# Syriac Text Generation Research

## Setup

### Import necessary libraries

In [63]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time

### Reading the data

In [84]:
METADATA = pd.read_csv('src/metadata.csv')

# Read the text file
path_to_file = 'src/1.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#print(f'Length of text: {len(text)} characters')
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

# Vectorize the text
tokens = text.split()
chars = tf.strings.unicode_split(tokens, input_encoding='UTF-8')
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


# Create training examples and targets
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))


29 unique characters


## Build the Model

In [87]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 30

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                    return_sequences=True,
                                    return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
            x, states = self.gru(x, initial_state=states, training=training)
            x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try the Model

In [88]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    
model.summary()

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

print("Input:\n", text_from_ids(input_example_batch[0]).numpy().decode('utf-8'))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy().decode('utf-8'))

2023-08-17 16:44:24.879399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [17323]
	 [[{{node Placeholder/_0}}]]
2023-08-17 16:44:24.879827: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [17323]
	 [[{{node Placeholder/_0}}]]


Model: "my_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  900       
                                                                 
 gru_4 (GRU)                 multiple                  3244032   
                                                                 
 dense_4 (Dense)             multiple                  30750     
                                                                 
Total params: 3,275,682
Trainable params: 3,275,682
Non-trainable params: 0
_________________________________________________________________
Input:
 ܡ ܕܟܠܐ ܡܬܢܣܒ ܠܗ ܡܢܗ<EOS> ܡܢ ܕܫܩܠ ܓܝܪ ܒܛܝܒܘܬܐ ܐܦ ܒܛܝܒܘܬܐ ܘܠܐ ܠܗ<EOS>ܕܢܬܠ<EOS> ܘܐܢܬ ܚܒܝܒܝ ܕܫܐܠܬܢܝ ܐܝܟ 

Next Char Predictions:
 ܥܡSܬܫܟ[UNK]̈ܚܫ̈ܓ<ܒܕ ܨܠܙܩܚܫܗ>ܠܛܪܨSSܣܦOܛܚܝܦ>ܫܠܒܫܩ̈ܓܢ<̈ܩܥܡܥSܐܕܕܗܣOܐܪܘܘܦEܢܙܩܐܠOܙܠܘܗܓܢܝEܫܘOܡ̈ܪܪܚܙܘܬܛܗܚ>Oܒܪ<ܗܨ


## Train the Model

In [89]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()

# Attach an optimizer and a loss function
model.compile(optimizer='adam', loss=loss)  

# Configure checkpoints
checkpoint_dir = './training_checkpoints'   # Directory where the checkpoints will be saved
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")    # Name of the checkpoint files

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Execute Training
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Prediction shape:  (64, 100, 30)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.4009912, shape=(), dtype=float32)
Epoch 1/2


2023-08-17 16:44:29.335001: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [17323]
	 [[{{node Placeholder/_0}}]]
2023-08-17 16:44:29.335332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [17323]
	 [[{{node Placeholder/_0}}]]
2023-08-17 16:44:29.407589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' 

2/2 [==============================] - 2s 770ms/step - loss: 3.3940
Epoch 2/2
2/2 [==============================] - 1s 727ms/step - loss: 3.3422


## Generate text

In [90]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [91]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [92]:
start = time.time()
states = None
next_char = tf.constant(['ܟܐܦܐ'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

2023-08-17 16:44:40.770911: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-17 16:44:40.771461: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-17 16:44:40.772163: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

ܟܐܦܐܬܕܥܟEܓܝܡܢܙܟܘܫܨ<ܩܡܟܡܐܓܥ ܦ>ܒܐܗܛܛܦܫܠܨ<̈ܓܗܕ ̈ܝܨܩ<ܢܛܓܢEܢܘܡܦܗܗOܙܘܛܗܐܬܒܗ̈ܝܢܛ̈ܝܡܘ ܠܪܙܡSܣ ܒܥܛܥܫܛܡܝܝܠEܘSܒܠܚ̈ ܪܗܕSܬܬܛ<̈ܟܓܓܝܕܟܛܩܢܐܟܪܝ ܕܡܚܚܬܐܒܚܚܫSܩܠܥEܐܛܘܚܐOܟܟܥܠܟܓܙSܡܗܐܠܝܘܕܫܚOܪܫ̈Oܢܚ>> Oܢܐܡܓ ܠOܥܨE<ܗܗܕܣܓܦ<ܛSܡ̈SܗܛܕS̈ ܝܐ>ܬ<<<ܡܨܘS̈̈ܕ<ܒ ܓܗܣܡܪ̈ ܪܬܣܫËEܝܐܗܠܝܩܓܒܙܠܢܙܝܠܢܩܩܛܘܝܒܨܐܓܡܘܕܣܩEEܠܘܥܦܒܫܚܬܣ<ܝܕܝܥܦEܙܡ ܣܝSܝOܫܕ>ܗܙܘܪOܕܣ>ܙܨܓܗܙܦO ܟ>ܨܬܥܟܥܙ̈< ܢܢܥܪܗܓܡSܫܙܥ̈<ܘ>>ܨܥܦܠ>SSܗܕOEܥܘ>ܣܕܘܛ̈ܣܣܠ>SEܟܗܫܢܣܨܓܐܦ<ܪEܨܕܚ<ܕܠ<ܠܢ>S ܕܡܠܕܐܙ>Oܠ<ܘ><ܗE <ܛܙܫܓܒܒ>ܡܗܟܥ̈ܠܐܢܫܝܬܓܝ̈ܐܕܡܫܦܝܒ EܛܒܐOܫ ܥܕܠܠ̈ܘܙ Oܓܟܫ>ܪܪܛܟܚܡܡSܐܒܙܛܝܢܙSܚܗܘ<ܛSܡܕܪܠܕ̈ܢܣܐSܪܡܗܪܬܐܙܦ ܣܪܣܣEܟܗEܬSܨ̈ܓܟܣܠܥ̈ܗܗܐܥܝܐܗܓܨܣܒܬ̈ܝܓܡܡܝܫܬܚ̈ܬܟ<ܫܙܡ<ܘܗܡܗ<ܦܛܦܥ ܩ̈ܕܝ>ܛܬ ܚܗܒܒܢܩEEܦܙܗܐܟܗ<ܝܙܛܡܕ ܪܒܙܠܣܩܥ ܗܪܡ<EܛܐܕܐܟܡܕOܨܚܣOܨܨܝEEܪ ܓܦ< ܦ̈̈ܫܛEܣܒO ̈SܫܢܡSܘOܚ<ܟܗܐܨS̈ܪܛܢEܚܣܓܨܙܫSܐܩܚܕܙSܥܘ ܕܠܠ>ܝܨܙܗܘ ܦܥܡܕܨ>>ܐܩܩܦܗܘܨܫܡܢO<ܛEܬܢ>ܙܙEE<ܣܢܡ Eܬܒܢܥ ܚOܒ>ܘܐܩܘܛܡ ܗܫܨܟܣܩ̈OEܠܬܫ<ܐܠܬܛܢܪܥܫܣܟܫܫܘܦܒܐܦܚ>Eܘܒܚܦܓܟܝܕ ܐܫ< ܘܪܘ >ܨܝܛܗܠܒܥSܩܐܚܣܨܛܢܛܐܪܩSܫܟܛ̈̈ܙܙܬ̈ܡܗܚEܐܡܒܚܛܠ̈ܚ>ܡܛܕܟܒܬܢܣܠܙܕܥܢܪܫܐܕܕܟE>ܓܥܢܬܛܦܢܓܐ<ܕܫܦܒܟܒܓܠܐܚܘܓEܟ>ܫܨܐܡܛܕܝܠܓܓܗܐܡܩܟSSܝ>ܪܘܩE<S ܦܕ ܫܬܡ  ܛܣܘܒ̈ܐܒܘܥܗ̈Eܣܥ ܨܡ̈ܪܝܗEܚܓܬ̈Oܛܩܪ ܬܘܗ̈>ܪܦܥܢOܡܨܢܐܙܫSܛܪ<ܥ̈ܕܚEܢܐܦEܘܥܘܡEܓܓܠܬܓܕܟܢ ܦܘ̈Eܚܓ Oܓܨ<ܝܨܝܗܒܫܘܨܚܒܒ ܓܫܟܣ <ܐ ܨܢ

## Exporting the Generator

In [ ]:
# Saving the Model
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

# Quick Checking with the Model
states = None
next_char = tf.constant(['ܗܕܐ'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))